In [10]:
import os, sys
import jieba, codecs, math
import jieba.posseg as pseg
import pickle

In [11]:
names = {}
relationships = {}
lineNames = []

path = '/dataset/SNA_final/'
ff = path+"三國001.txt"
dd = path+"all_dict.txt"

In [14]:
all_name = pickle.load(open(path+'all_name.pkl', 'rb'))
all_2ndname = pickle.load(open(path+'all_2ndname.pkl', 'rb'))

In [15]:
jieba.load_userdict(dd)
with codecs.open(ff.encode('utf-8'), "r", "utf8") as f:
    for line in f.readlines():
        poss = pseg.cut(line)
        lineNames.append([])
        for w in poss:
            if w.flag != "nr" or len(w.word) < 2:
                continue
            lineNames[-1].append(w.word)
            if names.get(w.word) is None:
                names[w.word] = 0
                relationships[w.word] = {}
            names[w.word] += 1

In [16]:
for name, times in names.items():
    print(name, times)

曹嵩 3
束草 1
童顏 1
若巨雷 1
張曰 1
蹇碩 2
玄德兵 1
若萌 1
溫德殿 2
侯覽 1
程曠 1
董卓 5
張名飛 1
關羽 2
玄德家 1
玄孫 1
關公 1
張世平 1
天公 2
帝尊信 1
劉焉 9
黃巾 3
汝得之 1
聞中郎 1
靖王 1
玄德領 2
玄德幼 1
玄德 21
蒼天 1
劉貞 1
桑村 1
張角 13
山后 1
劉雄 1
玄德見 1
玄德遂 1
漢景帝 1
阿父 1
高祖 1
何進 2
漢魯恭 1
安民 2
唐周 2
盧植 10
大賢良 2
帝覽 1
竇武 2
別二客 1
關名羽 1
張來 1
梁上飛 1
汝中風 1
張讓 2
漢武 1
寇發 1
張飛 9
玄德祖 1
程遠志 5
張引 1
左有雲 1
玄德回 1
雲長齊 1
盧中郎 1
於桓 1
廣宗來 1
立功 2
張梁 8
廣宗 3
玄德謂 1
玄德揮 1
龔景 3
夏侯氏 1
王之後 1
玄德大 1
於玉堂 1
玄德引 3
南華老仙 1
丹鳳 1
郎蔡邕 1
於桃園 1
干政之 1
武中興 1
鄭玄 1
皇甫嵩 5
莊田 1
玄德甚 1
玄德軍 1
桓帝 2
陳蕃 2
花開正 1
雲長 1
左豐 2
公孫瓚 1
馬匹 1
玄德曰 8
鄒靖 8
巨鹿郡 1
張縱馬 1
夏惲 1
曹節 3
劉元起 1
玄德摩軍 1
黃巾蓋 1
於園 1
鄧茂 3
張寶 8
雲長造 1
童童 1
孟德 1
郭勝 1
桑樹 1
靈帝 1
馬引軍 1
弘曾 1
公看榜而 1
曹騰 1
劉弘 1
玄德謝 1
明公宜 1
劉備 1
蘇雙 1
都尉 2
曉夜 1
玄德年 1
金失侯 1
劉勝 2
張天 1
汝可引 1
馬元義 2
侍奉 1
許劭 1
趙忠 1
大吉 1
劉名備 1
秦滅 1
雲長刀 1


In [8]:
for line in lineNames:                    #對於每一段
    for name1 in line:                    
        for name2 in line:                # 每段中的任意兩個人
            if name1 == name2:
                continue
            if relationships[name1].get(name2) is None:        # 若兩人尚未同時出現則新建項
                relationships[name1][name2]= 1
            else:
                relationships[name1][name2] = relationships[name1][name2]+ 1        # 兩人共同出現次數加 1

In [9]:
relationships

{'丹鳳': {'張飛': 1,
  '於園': 1,
  '玄德': 3,
  '玄德遂': 1,
  '花開正': 1,
  '關名羽': 1,
  '雲長': 1,
  '雲長齊': 1},
 '侍奉': {'南華老仙': 1,
  '大吉': 1,
  '大賢良': 2,
  '天公': 1,
  '巨鹿郡': 1,
  '張寶': 2,
  '張梁': 2,
  '張角': 6,
  '曉夜': 1,
  '汝得之': 1,
  '皇甫嵩': 1,
  '盧植': 1,
  '立功': 1,
  '童顏': 1,
  '若萌': 1,
  '蒼天': 1,
  '馬元義': 2,
  '黃巾': 1},
 '侍曹騰': {'夏侯氏': 1,
  '孟德': 1,
  '張寶': 4,
  '張梁': 4,
  '曹嵩': 2,
  '曹嵩生': 1,
  '汝中風': 1,
  '皇甫嵩': 1,
  '許劭': 1,
  '都尉': 2,
  '馬匹': 1},
 '侯覽': {'夏惲': 1,
  '帝尊信': 1,
  '帝覽': 1,
  '干政之': 1,
  '張讓': 2,
  '於玉堂': 1,
  '曹節': 2,
  '梁上飛': 1,
  '溫德殿': 2,
  '程曠': 1,
  '趙忠': 1,
  '郎蔡邕': 1,
  '郭勝': 1,
  '阿父': 1},
 '公孫瓚': {'劉元起': 1,
  '劉勝': 2,
  '劉名備': 1,
  '劉弘': 1,
  '劉貞': 1,
  '劉雄': 1,
  '弘曾': 1,
  '桑村': 1,
  '桑樹': 1,
  '漢景帝': 1,
  '漢武': 1,
  '玄孫': 1,
  '玄德': 2,
  '玄德家': 1,
  '玄德幼': 1,
  '玄德祖': 1,
  '盧植': 1,
  '童童': 1,
  '鄭玄': 1,
  '金失侯': 1,
  '靖王': 1},
 '公看榜而': {'劉焉': 1,
  '安民': 1,
  '張名飛': 1,
  '玄德': 1,
  '玄德回': 1,
  '玄德年': 1,
  '玄德曰': 1,
  '玄德甚': 1,
  '若巨雷': 1,
  '莊田': 1},
 '別二客': {'劉焉': 3,


In [11]:
with codecs.open(ff.encode('utf-8'), "r", "utf-8") as f:
    for line in f.readlines():
        print(line)
        print(1)

﻿第001回 宴桃園豪傑三結義 斬黃巾英雄首立功

1
　　滾滾長江東逝水，浪花淘盡英雄。是非成敗轉頭空。

1


1
　　青山依舊在，幾度夕陽紅。白髮漁樵江渚上，慣

1


1
　　看秋月春風。一壺濁酒喜相逢。古今多少事，都付

1


1
　　笑談中——

1


1
　　調寄《臨江仙》

1


1
　　話說天下大勢，分久必合，合久必分。週末七國分爭，併入于秦。及秦滅之後，楚、漢分爭，又併入於漢。漢朝自高祖斬白蛇而起義，一統天下，後來光武中興，傳至獻帝，遂分為三國。推其致亂之由，殆始於桓、靈二帝。桓帝禁錮善類，崇信宦官。及桓帝崩，靈帝即位，大將軍竇武、太傅陳蕃共相輔佐。時有宦官曹節等弄權，竇武、陳蕃謀誅之，機事不密，反為所害，中涓自此愈橫。

1


1
　　建寧二年四月望日，帝禦溫德殿。方升座，殿角狂風驟起。只見一條大青蛇，從梁上飛將下來，蟠於椅上。帝驚倒，左右急救入宮，百官俱奔避。須臾，蛇不見了。忽然大雷大雨，加以冰雹，落到半夜方止，壞卻房屋無數。建寧四年二月，洛陽地震；又海水泛溢，沿海居民，盡被大浪捲入海中。光和元年，雌雞化雄。六月朔，黑氣十餘丈，飛入溫德殿中。秋七月，有虹現於玉堂；五原山岸，盡皆崩裂。種種不祥，非止一端。帝下詔問群臣以災異之由，議郎蔡邕上疏，以為-墮雞化，乃婦寺干政之所致，言頗切直。帝覽奏歎息，因起更衣。曹節在後竊視，悉宣告左右；遂以他事陷邕於罪，放歸田裡。後張讓、趙忠、封-、段-、曹節、侯覽、蹇碩、程曠、夏惲、郭勝十人朋比為奸，號為“十常侍”。帝尊信張讓，呼為“阿父”。朝政日非，以致天下人心思亂，盜賊蜂起。

1


1
　　時巨鹿郡有兄弟三人，一名張角，一名張寶，一名張梁。那張角本是個不第秀才，因入山采藥，遇一老人，碧眼童顏，手執藜杖，喚角至一洞中，以天書三卷授之，曰：“此名《太平要術》，汝得之，當代天宣化，普救世人；若萌異心，必獲惡報。”角拜問姓名。老人曰：“吾乃南華老仙也。”言訖，化陣清風而去。角得此書，曉夜攻習，能呼風喚雨，號為“太平道人”。中平元年正月內，疫氣流行，張角散施符水，為人治病，自稱“大賢良師”。角有徒弟五百余人，雲遊四方，皆能書符念咒。次後徒眾日多，角乃立三十六方，大方萬餘人，小方六七千，各立渠帥，稱為將軍；訛言：“蒼天已死，黃天當立；歲在甲子，天下大吉。”令人各以白土書“甲子”二字於家中大門上。青、幽、徐、